<a href="https://colab.research.google.com/github/SrivenkateswaraRao/Research-Project/blob/master/CNN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from os import listdir
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import numpy as np

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import files
files.upload() 

In [ ]:
import CreatePestImageData_AlexNetCopy as Imagedataset
import SupportingFunctions as support

In [ ]:
support.DownLoadFiles('1aXb25Wp9sG7mL6AQgYY9ElrGcERi95JU',drive)

In [13]:
support.Unzip('/content/GREENPATROL_DB_V3.0.zip','/content/Pests/Images')

In [15]:
support.fileCount('/content/Pests/Images')

4330


In [16]:
dataSet = Imagedataset.createImageDataset()

In [17]:
class_object = {0 : 'egg_wf' ,
                1 : 'egg_bt', 2 : 'egg_ta', 3 : 'wf', 4 : 'bt', 5 : 'ta', 6 : 'ta_maps', 7 : 'ta_larvae', 8 : 'ta_maps+larvae', 9 : 'tomato'}

In [ ]:
dataSet.prepare_dataset('/content/Pests/Labels' , class_object ,48, '/content/Pests/Images/',  test_size=11241 )

In [20]:
(trainY , trainX) , (testY , testX ) = (dataSet.trainY , dataSet.trainX ) , (dataSet.testY , dataSet.testX )


In [59]:
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
visible = Input(shape=(48,48,3))
conv1 = Conv2D(32, kernel_size=4, activation='relu')(visible)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(16, kernel_size=4, activation='relu')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
flat = Flatten()(pool2)
hidden1 = Dense(10, activation='relu')(flat)
output = Dense(10, activation='softmax')(hidden1)
model = Model(inputs=visible, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [60]:
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, 48, 48, 3)         0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 45, 45, 32)        1568      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 22, 22, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 19, 19, 16)        8208      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 9, 9, 16)          0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 1296)              0         
_________________________________________________________________
dense_27 (Dense)             (None, 10)                1297

In [67]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(44964, 32, 32, 3)
(44964,)
(11241, 32, 32, 3)
(11241,)


In [68]:
filepath="/content/drive/My Drive/Models/CNN2/weights-{epoch:02d}-{accuracy:.3f}.hdf5"
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath, monitor='accuracy',
                               verbose=1, mode='max',save_best_only=True)

callbacks_list = [checkpoint]

In [62]:
from keras.utils import to_categorical
trainY = to_categorical(trainY)
testY = to_categorical(testY)

In [70]:
model.fit(trainX,trainY,verbose=1,epochs=50, validation_data= (testX,testY), callbacks=[checkpoint])

Train on 44964 samples, validate on 11241 samples
Epoch 1/50
44964/44964 [==============================] - 25s 562us/step - loss: 0.8013 - accuracy: 0.6770 - val_loss: 0.8577 - val_accuracy: 0.6603

Epoch 00001: accuracy improved from -inf to 0.67701, saving model to /content/drive/My Drive/Models/CNN2/weights-01-0.677.hdf5
Epoch 2/50
44964/44964 [==============================] - 26s 584us/step - loss: 0.8003 - accuracy: 0.6778 - val_loss: 0.8501 - val_accuracy: 0.6610

Epoch 00002: accuracy improved from 0.67701 to 0.67783, saving model to /content/drive/My Drive/Models/CNN2/weights-02-0.678.hdf5
Epoch 3/50
44964/44964 [==============================] - 26s 568us/step - loss: 0.7966 - accuracy: 0.6774 - val_loss: 0.8562 - val_accuracy: 0.6623

Epoch 00003: accuracy did not improve from 0.67783
Epoch 4/50
44964/44964 [==============================] - 25s 547us/step - loss: 0.7949 - accuracy: 0.6793 - val_loss: 0.8514 - val_accuracy: 0.6633

Epoch 00004: accuracy improved from 0.6778